In [1]:
import os
import fcwt
import matplotlib.pyplot as plt
import numpy as np

## Import custom functions from 'pre_processing.py' ##
try:
    import importlib
    importlib.reload(pp)
except NameError: # It hasn't been imported yet
    import pre_processing as pp
    
fnirs_data = r'C:\Users\61407\mental-health-modelling\data\fNIRS\bandpass_filtered_whole_data'
scalogram_dir = r'C:\Users\61407\mental-health-modelling\fNIRS Mental Workload\scalograms'
all_data = pp.upload_all_subjects(fnirs_data, scaled=True)

In [2]:
from ssqueezepy import ssq_cwt, ssq_stft
from ssqueezepy.experimental import scale_to_freq
#%%# With units #######################################
os.environ['SSQ_GPU'] = '1'
from ssqueezepy import Wavelet, cwt, stft, imshow
import torch

In [15]:
def extract_scalogram_tensors(array_1d):
    wavelet = Wavelet('morlet')
    Wx, scales = cwt(array_1d, wavelet)
    return Wx

In [17]:
sub_groups = all_data.groupby(all_data.index.get_level_values(0))
for sub in sub_groups.groups.keys():
    sub_data = sub_groups.get_group(sub)
    data_blocks, block_change_idx = pp.detect_block_changes(sub_data, 'difficulty')
    label_dict = {block:int(df.difficulty.unique()) for block, df in data_blocks.items()}
    np.save(os.path.join(scalogram_dir, f'{sub}_labels.npy'), label_dict, allow_pickle=True)
    
    array_3d = np.array([x.drop('difficulty', axis=1).to_numpy().T for x in data_blocks.values()])
    tensor_3d = torch.tensor(array_3d, dtype=torch.float32).cuda()
    tensor_4d = torch.stack([torch.stack([extract_scalogram_tensors(tensor_3d[i, j, :]) for j in range(tensor_3d.shape[1])]) for i in range(tensor_3d.shape[0])], dim=0)
    torch.save(tensor_4d, os.path.join(scalogram_dir, f'{sub}_scalogram_tensors.pt'))